In [2]:
import os, sys
from pathlib import Path

src_path = Path(os.getcwd()).resolve().parents[1]  
sys.path.append(str(src_path))

In [ ]:
from langchain.chat_models import init_chat_model
from enums.LLM import LLM
from prompts.promptmanager import PromptManager
from prompts.prompt_type.fairy.FairyPromptType import FairyPromptType
PromptManager(FairyPromptType.FAIRY_GUILD_SYSTEM).get_prompt()
llm = init_chat_model(model = LLM.GPT4_1_MINI, temperature=0)

from core.common import get_inventory_items
